In [1]:
import pandas as pd
import numpy as np
import nltk
import string
import re
import emoji
from gensim.models import Word2Vec
import torch
import json
import time

In [2]:
# getting the twitter comments
DATA_PATH = '../data/twitter-data-cleaned.txt'
with open(DATA_PATH, 'r',  encoding="utf-8") as f:
    data = f.readlines()
print('Tweets')
print(data[:3])

# getting our chosen emojis
SELECTED_EMOJIS_PATH = '../data/best-emojis.json'
with open(SELECTED_EMOJIS_PATH, 'r') as f:
    EMOJIS = json.load(f)
EMOJI_CHARS = [e['char'] for e in EMOJIS]
print('Chosen emojis')
print(EMOJI_CHARS)

Tweets
['Y’all can’t get tender when I don’t respond 😂\n', 'dawg this game finna blow me 😂\n', '😂 😂 😂\n']
Chosen emojis
['😂', '😍', '😭', '😊', '💕', '😒', '😉', '👌', '👍', '🙏', '👀', '🔥', '💯', '👏', '💪']


In [4]:
# preprocessing the data
from parsing import Tokenizer, TokenType, Token
tokenizer = Tokenizer(EMOJI_CHARS)
# take 3 previous words as context for the emoji
context = {e:[] for e in EMOJI_CHARS}
for tweet in data:
    tokens = tokenizer.tokenize(tweet)
    for i,token in enumerate(tokens):
        if token.token_type == TokenType.EMOJIS:
            closest = tokenizer.findClosest3Words(tokens, i)
            if closest:
                context[token.raw].append(closest)

for e, words in context.items():
    print(e)
    print(len(words))
    print(words[:3])

😂
192
[['i', 'don’t', 'respond'], ['finna', 'blow', 'me'], ['we', 'in', 'trouble']]
😍
153
[['scramble', 'city', 'ksw55'], ['just', 'wow', 'amazing'], ['the', 'second', 'pic']]
😭
164
[['miss', 'paris', 'tho'], ['they’re', 'so', 'cuteee'], ['they’re', 'so', 'cuteee']]
😊
114
[['hi', 'can', 'u'], ['all', 'of', 'them'], ['harming', 'other', 'people']]
💕
133
[['olivia', 'newton', 'john'], ['amp', 'baby', 'gia'], ['do', 'the', 'job']]
😒
98
[['starters', 'on', 'defense'], ['talk', 'to', 'em'], ['know', 'the', 'feeling']]
😉
109
[['gets', 'literally', '25000+'], ['amp', 'gurlish', 'side'], ['you’re', 'right']]
👌
110
[['make', 'people', 'laugh'], ['thanks'], ['war', 'here', 'soon']]
👍
123
[['it', 'up', 'brother'], ['thanks', 'venting', 'to'], ['you’re', 'very', 'welcome']]
🙏
148
[['the', 'shirt', 'please'], ['to', 'see', 'again'], ['my', 'dear', 'friend']]
👀
123
[['when', 'life', 'gets'], ['rt', 'that', 'tweet'], ['one', 'is', 'from']]
🔥
190
[['install', 'golden', 'match'], ['this', 'gorgeous', '